In [32]:
library(tidyverse)
library(tidymodels)
library(repr)
library(rvest)
library(stringr)
library(dbplyr)
library(DBI)
library(RPostgres)
library(lubridate)
options(repr.matrix.max.rows = 6)

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3      ✔ yardstick 0.0.7 

Warning message:
“package ‘broom’ was built under R version 4.0.2”
Warning message:
“package ‘dials’ was built under R version 4.0.2”
Warning message:
“package ‘infer’ was built under R version 4.0.3”
Warning message:
“package ‘modeldata’ was built under R version 4.0.1”
Warning message:
“package ‘parsnip’ was built under R version 4.0.2”
Warning message:
“package ‘recipes’ was built under R version 4.0.1”
Warning message:
“package ‘tune’ was built under R version 4.0.2”
Warning message:
“package ‘workflows’ was built under R version 4.0.2”
Warning message:
“package ‘yardstick’ was built under R version 4.0.2”
── Conflicts ────

# Algerian Forest Fires and the Weather
## Background and Introduction:
As our climate changes, and forest fires become more of a danger to humans all around the world, it is more and more important to understand the conditions which can lead to their ignition. Due to increasing temperatures, summer temperatures have been increasing, resulting in dryer heat (lower humidity), dryer forests, which lead to easier ignition. Rising temperatures result in the the unequality of temperature between the sky and the earth, leading to further likelihood of lightning--along with human campfires, cigarettes, and fireworks alike, these igniting forces are sure to lead to forest fires. ( 


## Our research question:  What are the weather conditions needed to result in a forest fire in the Algerian Forest Areas?
## The dataset:
Our dataset, "The Algerian Forest Fires Dataset" [link to dataset] (https://archive.ics.uci.edu/ml/datasets/Algerian+Forest+Fires+Dataset++) provides information on forest fires in two areas: the Sidi-Bel Abbas and Bejaia regions. The data was taken from June 2012 to September 2012, and contains information on the following data:

1. Date: (DD/MM/YYYY) Day, month ('june' to 'september'), year (2012)
Weather data observations
2. Temp: temperature noon (temperature max) in Celsius degrees: 22 to 42
3. RH: Relative Humidity in %: 21 to 90
4. Ws: Wind speed in km/h: 6 to 29
5. Rain: total day in mm: 0 to 16.8
FWI Components
6. Fine Fuel Moisture Code (FFMC) index from the FWI system: 28.6 to 92.5
7. Duff Moisture Code (DMC) index from the FWI system: 1.1 to 65.9
8. Drought Code (DC) index from the FWI system: 7 to 220.4
9. Initial Spread Index (ISI) index from the FWI system: 0 to 18.5
10. Buildup Index (BUI) index from the FWI system: 1.1 to 68
11. Fire Weather Index (FWI) Index: 0 to 31.1
12. Classes: two classes, namely Fire and Not Fire

Vector 1 is the date, 2-5 are all weather based observations, vectors 6-11 are all "Fire Weather Index" components, and vector 12 is whether there was a fire or not in the area.
Fire Weather Index, or FWI, is a general measure used to predict the intensity of a potential fire in the area, and takes into account each component when calculating this. 

In [39]:

# Title: Factors Contributing to Algerian Forest Fires
# Introduction:
forest_fires_untidy <- read_csv(file = "https://archive.ics.uci.edu/ml/machine-learning-databases/00547/Algerian_forest_fires_dataset_UPDATE.csv", 
                                skip =1, col_names = TRUE, col_types = cols_only(
                                      day = col_integer(),
                                      month = col_integer(),
                                      year = col_integer(),
                                      Temperature = col_double(),
                                      RH = col_double(),
                                      Ws = col_double(),
                                      Rain = col_double(),
                                      FFMC = col_double(),
                                      DMC = col_double(),
                                      DC = col_double(),
                                      ISI = col_double(),
                                      BUI = col_double(),
                                      FWI = col_double(),
                                      Classes = col_character()))  
  
bejaia_region <- forest_fires_untidy %>%
    slice(1:122) %>%
    mutate(region = "bejaia")

sidi_bel_region <- forest_fires_untidy %>%
    slice(125:246) %>%
    mutate(region = "sidi bel abbes")

forest_fire_data <- rbind(bejaia_region, sidi_bel_region) 

fire_data <- forest_fire_data %>%
     select(day, month, Temperature, FWI, Classes)

fire_data

set.seed(1)
fire_split <- initial_split(fire_data, prop = 0.75, strata = Classes)
fire_train <- training(fire_split)
fire_test <- testing(fire_split)

fire_recipe <- recipe(Classes ~ month + Temperature + FWI, data = fire_train) %>%
    step_scale(all_predictors()) %>%
    step_center(all_predictors())

set.seed(1)
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 3) %>%
    set_engine("kknn") %>%
    set_mode("classification")

knn_fit <- workflow() %>%
    add_recipe(fire_recipe) %>%
    add_model(knn_spec) %>%
    fit(data = fire_train)

fire_test_predictions <- predict(knn_fit, fire_test) %>%
    bind_cols(fire_test)


Warning message:
“18 parsing failures.
row   col   expected                        actual                                                                                                       file
123 day   an integer Sidi-Bel Abbes Region Dataset 'https://archive.ics.uci.edu/ml/machine-learning-databases/00547/Algerian_forest_fires_dataset_UPDATE.csv'
123 NA    14 columns 1 columns                     'https://archive.ics.uci.edu/ml/machine-learning-databases/00547/Algerian_forest_fires_dataset_UPDATE.csv'
124 day   an integer day                           'https://archive.ics.uci.edu/ml/machine-learning-databases/00547/Algerian_forest_fires_dataset_UPDATE.csv'
124 month an integer month                         'https://archive.ics.uci.edu/ml/machine-learning-databases/00547/Algerian_forest_fires_dataset_UPDATE.csv'
124 year  an integer year                          'https://archive.ics.uci.edu/ml/machine-learning-databases/00547/Algerian_forest_fires_dataset_UPDATE.csv'
... ..... ...

day,month,Temperature,FWI,Classes
<int>,<int>,<dbl>,<dbl>,<chr>
1,6,29,0.5,not fire
2,6,29,0.4,not fire
3,6,26,0.1,not fire
⋮,⋮,⋮,⋮,⋮
28,9,27,0.2,not fire
29,9,24,0.7,not fire
30,9,24,0.5,not fire


# Methods
For our analysis, we will not be looking at any of the individual FWI components and instead will be looking at the weather data observations. We are not interested in the potential of intensity, rather we are interested in the contribution of the weather in forest fire ignition.
Furthermore, as the data was all taken from a single year, we do not select the year column as this information is redundant.

# Expected outcomes and significance:
We plan to better predict what weather conditions will lead to a fire.